# Questão 2

## Implementação do Perceptron de Múltiplas Camadas

Implemente uma rede perceptron de múltiplas camadas treinando-a com os seguintes
algoritmos:

    a) algoritmo da retropropagação em modo estocástico usando a regra delta;
    b) algoritmo da retropropagação em modo por lote usando a regra delta;
    c) algoritmo da retropropagação usando a regra delta com termo do momento;

## Importações utilizadas

In [77]:
from numpy import exp, array, random, dot, shape, zeros, transpose, matrix
from math import ceil, sinh, cosh
#import matplotlib.pyplot as plt
#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score

## Classes desenvolvidas

In [78]:
class NeuronLayer():
    """
    Classe representando a camada de neurônios da rede
    Parâmetros:
        - num_neurons: o número de neurônios na camada
        - num_inputs: o número de entradas que cada neurônio da camada terá
    """
    def __init__(self, num_neurons, num_inputs):
        self.weights = 2 * random.random((num_inputs, num_neurons)) - 1
        
class NeuralNetwork():
    def __init__(self, layer1, layer2):
        self.layer1 = layer1
        self.layer2 = layer2

    def activation_func(self, func_name, z):
        """
        Executa uma das funções de ativação da rede
        Parâmetros:
            - func_name: nome da função de ativação
            - z: valor a ser aplicado na função de ativação
        """
        if(func_name == 'sigmoid'):
            return 1 / (1 + exp(-z))
        elif(func_name == 'tanh'):
            return sinh(z) / cosh(z)
        elif(func_name == 'relu'):
            if(z <= 0):
                return 0
            else:
                return z
        elif(func_name == 'sigmoid_deriv'):
            return z * (1-z)
        elif(func_name == 'relu_deriv'):
            if(z<=0):
                return 0
            else:
                return 1

    def train_network(self, inputs, expected_outs, num_iterations, learning_rate, backprop_type):
        """
        Método para treinamento da rede neural e ajuste dos pesos sinápticos
        Parâmetros:
            - inputs: o conjunto de treinamento
            - expected_outs: saídas desejadas do conjunto de treinamento
            - num_iterations: épocas (duração do treinamento)
            - learning_rate: taxa de aprendizado
            - backprop_type: o tipo de retropropagação a ser usado no treinamento ("momentum", "batch", "stochastic)
            
        """
        alpha = 0.1                                     #constante do momento (0 <= alpha < 1)
        samples, features = shape(inputs)
        layer1_adjustment = 0
        layer2_adjustment = 0
        
        for count in range(num_iterations):
            #print("época " + str(count + 1) + ":")

            if((backprop_type == "batch") or (backprop_type == "momentum")):
                out_l1, out_l2 = self.classify(inputs)
                
                ### Cálculo dos erros na camada 2
                layer2_error = expected_outs - out_l2
                layer2_grad = layer2_error * self.activation_func("sigmoid_deriv",out_l2)      # gradiente local do neurônio da camada de saída
                #print("erro camada 2: \n" + str(layer2_error))

                ## Com base no resultado da camada 2, calcular o erro na camada 1
                layer1_error = layer2_grad.dot(self.layer2.weights.T)
                layer1_grad = layer1_error * self.activation_func("sigmoid_deriv",out_l1)                 # gradiente da camada 1
                #print("erro camada 1: \n" + str(layer1_error))
                
                if(backprop_type == "momentum"):
                    if(count == 0):
                        layer1_adjustment = (inputs.T.dot(layer1_grad)) * learning_rate
                        layer2_adjustment = out_l1.T.dot(layer2_grad) * learning_rate
                    else:
                        ## Cálculo da regra delta com constante do momento
                        layer1_adjustment = (alpha * temp1) + ((inputs.T.dot(layer1_grad)) * learning_rate)
                        layer2_adjustment = (alpha * temp2) +(out_l1.T.dot(layer2_grad) * learning_rate)
                else:
                    ## Cálculo normal da regra delta
                    layer1_adjustment = (inputs.T.dot(layer1_grad)) * learning_rate
                    layer2_adjustment = out_l1.T.dot(layer2_grad) * learning_rate

                self.layer1.weights += layer1_adjustment
                temp1 = self.layer1.weights
                self.layer2.weights += layer2_adjustment
                temp2 = self.layer2.weights
            else:
                #treinamento estocástico
                for count2 in range(samples):
                    out_l1, out_l2 = self.classify(inputs[count2, :])
                    #print("saída camada 1: \n" + str(out_l1) + "\nsaída camada 2: \n" + str(out_l2))

                    ### Cálculo dos erros na camada 2
                    layer2_error = expected_outs[count2, :] - out_l2
                    layer2_grad = layer2_error * self.activation_func("sigmoid_deriv",out_l2)                 # gradiente local do neurônio da camada de saída

                    ## Com base no resultado da camada 2, calcular o erro na camada 1
                    layer1_error = layer2_grad.dot(self.layer2.weights.T)
                    layer1_grad = layer1_error * self.activation_func("sigmoid_deriv",out_l1)                 # gradiente da camada 1

                    ## Regra delta
                    ar = array([inputs[count2, :]])
                    l1_grad = array([layer1_grad])
                    o_l1 = array([out_l1])
                    l2_grad = array([out_l2])
                    
                    layer1_adjustment = (ar.T.dot(l1_grad)) * learning_rate
                    layer2_adjustment = o_l1.T.dot(l2_grad) * learning_rate

                    self.layer1.weights += layer1_adjustment
                    temp1 = self.layer1.weights
                    self.layer2.weights += layer2_adjustment
                    temp2 = self.layer2.weights

    def classify(self, inputs):
        """
        Classifica uma entrada pela rede neural
        Parâmetros:
            - inputs: valor de um conjunto de treinamento/classificação
        """
        output_from_layer1 = self.activation_func("sigmoid", (dot(inputs, self.layer1.weights)))
        output_from_layer2 = self.activation_func("sigmoid", (dot(output_from_layer1, self.layer2.weights)))
        return output_from_layer1, output_from_layer2

    def print_weights(self):
        print("Pesos da camada 1: ")
        print (self.layer1.weights)
        print("Pesos da camada 2: ")
        print (self.layer2.weights)

## Informações adicionais

Por questões de simplicidade, foi decidido a implementação de um perceptron com apenas duas camadas. Como pode ser visto no código abaixo, são criadas duas camadas de neurônios, sendo a primeira com 5 neurônios, recebendo as entradas do conjunto de treinamento, e a segunda contendo apenas 1 neurônio, recebendo como entrada as saídas dos neurônios da camada anterior.

O conjunto de treinamento contém 6 exemplos de entrada e 2 exemplos utilizados para treinar a classificação.

In [80]:
if __name__ == "__main__":
    LEARNING_RATE = 0.5

    #random.seed(1)

    # Rede neural de duas camadas (setando valores (numero de neurônios e seu numero de entradas))
    layer1 = NeuronLayer(5, 3)
    layer2 = NeuronLayer(1, 5)

    neural_network = NeuralNetwork(layer1, layer2)

    # Atribuição randomica de pesos a rede neural
    print ("Pesos antes de iniciar o treinamento: ")
    neural_network.print_weights()

    # Conjunto de treinamento (6 exemplos com 3 valores de entrada e 1 valor de saída).
    training_set_inputs = array([[0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
    training_set_outputs = array([[1, 1, 0, 1, 0, 1]]).T

    # treinamento da rede neural
    neural_network.train_network(training_set_inputs, training_set_outputs, 5000, LEARNING_RATE, "momentum")

    print ("Pesos de saída (após última iteração do treinamento): ")
    neural_network.print_weights()

    print("Saídas obtidas com relação aos conjuntos treinados: ")
    h, out3 = neural_network.classify(array([0, 1, 1]))
    h, out4 = neural_network.classify(array([1, 0, 1]))
    h, out5 = neural_network.classify(array([0, 1, 0]))
    h, out6 = neural_network.classify(array([1, 0, 0]))
    h, out7 = neural_network.classify(array([1, 1, 1]))
    h, out8 = neural_network.classify(array([0, 0, 0]))
    print(str(out3) + " " + str(out4) + " " + str(out5) + " " + str(out6) + " " + str(out7) + " " + str(out8))

    print ("Teste com entradas não treinadas: ")
    h, output = neural_network.classify(array([1, 1, 0]))
    print("[1, 1, 0] ==>" + str(output))
    h, out2 = neural_network.classify(array([0, 0, 1]))
    print ("[0, 0, 1] ==> " + str(out2))

Pesos antes de iniciar o treinamento: 
Pesos da camada 1: 
[[-0.96044121  0.54458214 -0.34804275 -0.90906084 -0.27221613]
 [ 0.02592391  0.31021019 -0.47048923  0.30738122  0.77587053]
 [ 0.20432253  0.20984768 -0.53915093  0.77970849  0.01007366]]
Pesos da camada 2: 
[[ 0.1281504 ]
 [ 0.29396554]
 [ 0.69110499]
 [-0.33849427]
 [ 0.06032128]]
Pesos de saída (após última iteração do treinamento): 
Pesos da camada 1: 
[[-8.04447826e+206  4.71504678e+206 -2.21195420e+206 -7.52860963e+206
  -2.27625560e+206]
 [-2.68084247e+205  1.86870369e+206 -5.25342650e+206  3.78318035e+206
   6.45255406e+206]
 [ 2.07611264e+206  2.17763540e+206 -3.90018438e+206  6.54853242e+206
   7.70100871e+204]]
Pesos da camada 2: 
[[ 2.20160601e+206]
 [ 2.76595425e+206]
 [ 6.99586373e+206]
 [-3.12483198e+206]
 [-1.08295430e+206]]
Saídas obtidas com relação aos conjuntos treinados: 
[1.] [1.] [0.] [1.] [0.] [1.]
Teste com entradas não treinadas: 
[1, 1, 0] ==>[1.]
[0, 0, 1] ==> [1.]


## Exemplos de saídas

Considerando a entrada 
    x = [0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]]

e a saída esperada
    y = [1, 1, 1, 0, 1, 0, 1]


### Regra delta com termo do momento (alpha = 0.1)


##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[-0.83475838  0.01699496 -0.42069627 -0.29071839  0.04908799]
     [ 0.35578725  0.95919087  0.93309106  0.1644563  -0.7621909 ]
     [-0.47051837  0.33232453  0.09078856 -0.21768878  0.30531074]]
 
Pesos da camada 2: 

    [[ 0.98271422]
     [-0.50859378]
     [-0.32569893]
     [ 0.85009907]
     [ 0.60985323]]
 
##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[-6.79963532e+206 -5.53665695e+205 -3.76593325e+206 -1.99586382e+206
       8.52225547e+205]
     [ 2.29986397e+206  8.62115036e+206  8.33225912e+206 -5.31630268e+205
      -6.78011909e+206]
     [-1.52776764e+206  2.27610798e+206  5.12704297e+205  6.02530888e+205
       3.72729736e+206]]
   
Pesos da camada 2: 

    [[ 7.10638320e+206]
     [-4.96690861e+206]
     [-3.98957106e+206]
     [ 7.17854279e+206]
     [ 7.50448908e+206]]
 
Saídas obtidas com relação aos conjuntos treinados:

    [1.] [1.] [0.] [1.] [0.] [1.]
    
Teste com entradas não treinadas: 

    [1, 1, 0] ==>[0.]
    [0, 0, 1] ==> [1.]
    
    
### Backpropagation por lotes

##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[ 0.99201534 -0.41928641 -0.36273008 -0.56463967 -0.33377834]
     [ 0.00518027  0.33828494 -0.93426758 -0.59924854 -0.39466139]
     [-0.06669957  0.89812476  0.71153876 -0.59760856 -0.86776041]]
 
Pesos da camada 2: 

    [[ 0.59587405]
     [-0.25582945]
     [ 0.88503957]
     [-0.38695687]
     [-0.37647607]]
 
##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[-0.48659945 -0.33860451  0.66946286 -0.87395413  0.6350526 ]
     [ 0.56707141 -0.77239012 -0.61720593  0.50100892  0.24499828]
     [ 0.3634765  -0.04300949 -0.15063007 -0.45175999  0.27899597]]
   
Pesos da camada 2: 

    [[ 4.46449034]
     [-5.26312244]
     [ 9.41891081]
     [ 1.98700465]
     [ 0.28427291]]
 
Saídas obtidas com relação aos conjuntos treinados (valores brutos, sem arredondamento):

    [0.97816022] [0.99820302] [0.01898581] [0.9848934] [0.01930171] [0.99570405]
    
Teste com entradas não treinadas:

    [1, 1, 0] ==>[0.02865417]
    [0, 0, 1] ==> [0.99925982]
    
    
### Backpropagation estocástico

##### Pesos antes de iniciar o treinamento:

Pesos da camada 1:

    [[-0.48248758 -0.33676161  0.67495536 -0.87249905  0.63841294]
     [ 0.58824616 -0.76286918 -0.59459496  0.50216722  0.26171083]
     [ 0.36356913 -0.042537   -0.15160516 -0.44412406  0.28095638]]
 
Pesos da camada 2: 

    [[ 0.66638244]
     [ 0.23120041]
     [ 0.90145531]
     [-0.65765234]
     [ 0.23953875]]
 
##### Pesos de saída (após última iteração do treinamento): 

Pesos da camada 1: 

    [[ 1.53124017 -2.07888298 -5.02242377 -0.16824009 -0.31320068]
     [-3.52113783  3.7862728  -2.65729379 -1.59201911 -0.31439086]
     [-0.94128309  1.73251232  5.00029858 -0.01472485 -0.84990957]]
   
Pesos da camada 2: 

    [[8286.4585554 ]
     [5460.94214015]
     [7328.32627216]
     [6109.32059047]
     [9544.06519478]]
 
Saídas obtidas com relação aos conjuntos treinados:

    [1.] [1.] [1.] [1.] [1.] [1.]
    
Teste com entradas não treinadas:

    [1, 1, 0] ==> [1.]
    [0, 0, 1] ==> [1.]
    
###### OBS.: Para as saídas no modo estocástico, o erro identificado está no gradiente da 1a camada, que diminui até se tornar 0 em certo ponto, influindo na saída e classificação errada das entradas.